## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# mu 101.5399 700

In [15]:
symbol = 'NVDA'

# 5 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '5m', '1mo', skip_dl=False) # skip_dl=True on redo's
print(f'{symbol} 5m Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 15 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '15m', '1mo', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 15m Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 30 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '30m', 'max', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 30m Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1h Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1h', '1y', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1h Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 5 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1d', '10y', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1d Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

NVDA 5m Interval Timestamp: 2024-10-09 17:57:02


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,down
kelly_1:2.5,0.3,0.448548,0.408095,0.415617
prob_up,0.125,0.002321,0.011855,0.061257
prob_static,0.375,0.391573,0.410935,0.35616
prob_down,0.5,0.606106,0.577211,0.582584
precision,0.75817,0.752508,0.768966,0.705645
recall,0.816901,0.792254,0.785211,0.616197
f1,0.786441,0.77187,0.777003,0.657895
support,"[276.0, 230.0, 284.0]","[276.0, 230.0, 284.0]","[276.0, 230.0, 284.0]","[276.0, 230.0, 284.0]"



NVDA 15m Interval Timestamp: 2024-10-09 17:57:06


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.3,0.541592,0.579181,0.565
prob_up,0.125,0.072679,0.041495,0.081642
prob_static,0.5,0.672566,0.699415,0.689285
prob_down,0.375,0.254755,0.25909,0.229072
precision,0.707865,0.703297,0.685393,0.588889
recall,0.863014,0.876712,0.835616,0.726027
f1,0.777778,0.780488,0.753086,0.650307
support,"[87.0, 73.0, 96.0]","[87.0, 73.0, 96.0]","[87.0, 73.0, 96.0]","[87.0, 73.0, 96.0]"



NVDA 30m Interval Timestamp: 2024-10-09 17:57:09


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,static,static
kelly_1:2.5,0.3,0.327455,0.679308,0.284055
prob_up,0.25,0.519611,0.210884,0.408031
prob_static,0.5,0.477587,0.770934,0.488611
prob_down,0.25,0.002802,0.018182,0.103358
precision,0.735294,0.775,0.7,0.540541
recall,0.78125,0.673913,0.65625,0.625
f1,0.757576,0.72093,0.677419,0.57971
support,"[46.0, 32.0, 45.0]","[46.0, 32.0, 45.0]","[46.0, 32.0, 45.0]","[46.0, 32.0, 45.0]"



NVDA 1h Interval Timestamp: 2024-10-09 17:57:11


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,static,static,static
kelly_1:2.5,0.3,0.883077,0.789254,0.725505
prob_up,0.125,0.025519,0.020046,0.072877
prob_static,0.375,0.916484,0.849467,0.803932
prob_down,0.5,0.057997,0.130487,0.123191
precision,0.793296,0.802005,0.759907,0.658349
recall,0.70297,0.864865,0.881081,0.927027
f1,0.745407,0.83225,0.81602,0.769921
support,"[212.0, 370.0, 202.0]","[212.0, 370.0, 202.0]","[212.0, 370.0, 202.0]","[212.0, 370.0, 202.0]"



NVDA 1d Interval Timestamp: 2024-10-09 17:57:15


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.825,0.780701,0.679814,0.536801
prob_up,0.875,0.843358,0.771295,0.669143
prob_static,0.0,0.095646,0.193787,0.232886
prob_down,0.125,0.060996,0.034918,0.097971
precision,0.846154,0.815217,0.829787,0.661376
recall,0.827957,0.806452,0.83871,0.672043
f1,0.836957,0.810811,0.834225,0.666667
support,"[186.0, 92.0, 189.0]","[186.0, 92.0, 189.0]","[186.0, 92.0, 189.0]","[186.0, 92.0, 189.0]"


## Sanity Checks

In [ ]:
df = f.load('NVDA','5m')
df.tail(3)

### Check Summary Output

In [ ]:
symbol='NVDA'
interval='5m'
period='1mo'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval, period)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')